<a href="https://colab.research.google.com/github/joony0512/Deep_Learning_Class/blob/main/Part6/P6_Ch01_CH02_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DL development pipeline

### Data Loader -> Model Definition -> Training Script
## Sequential


In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import activations

import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [ ]:
def get_sequential_model(input_shape):
    model = keras.Sequential(
        [
            # Input
            layers.Input(input_shape),

            # 1st Conv block
            layers.Conv2D(64, 3, strides = 1, activation = 'relu', padding ='same'), # padding ='same'으로 이미지 in, out 같은사이즈로 유지
            layers.Conv2D(64, 3, strides = 1, activation = 'relu', padding ='same'),
            layers.MaxPool2D(),
            layers.BatchNormalization(),
            layers.Dropout(0.5),

            # 2nd Conv block
            layers.Conv2D(128, 3, strides = 1, activation = 'relu', padding ='same'), # 위 block의 maxpooling으로 이미지 사이즈 절반으로 줄고 heuristic하게 filter size 2배로 올림
            layers.Conv2D(128, 3, strides = 1, activation = 'relu', padding ='same'),
            layers.MaxPool2D(),
            layers.BatchNormalization(),
            layers.Dropout(0.3),

            # Classfier
            layers.GlobalMaxPool2D(), # 필터별 가장 큰값 반환 -> 필터수만큼 대표값 반환
            layers.Dense(128, activation = "relu"),
            layers.Dense(1, activation = "sigmoid"), # 이진분류

        ]
    )
    return model

input_shape = (256, 256, 3)
model = get_sequential_model(input_shape)

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 256, 256, 64)      1792      
                                                                 
 conv2d_5 (Conv2D)           (None, 256, 256, 64)      36928     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 128, 128, 64)      0         
 g2D)                                                            
                                                                 
 batch_normalization (Batch  (None, 128, 128, 64)      256       
 Normalization)                                                  
                                                                 
 dropout (Dropout)           (None, 128, 128, 64)      0         
                                                                 
 conv2d_6 (Conv2D)           (None, 128, 128, 128)     7

## Fuctional API

In [ ]:
def get_functional_model(input_shape):
    inputs = keras.Input(input_shape)

    # 1st Conv block
    x = layers.Conv2D(64, 3, strides = 1, activation = 'relu', padding ='same')(inputs)
    x = layers.Conv2D(64, 3, strides = 1, activation = 'relu', padding ='same')(x)
    x = layers.MaxPool2D()(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.5)(x)

    # 2nd Conv block
    x = layers.Conv2D(128, 3, strides = 1, activation = 'relu', padding ='same')(x)
    x = layers.Conv2D(128, 3, strides = 1, activation = 'relu', padding ='same')(x)
    x = layers.MaxPool2D()(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.3)(x)

    # Classfier
    x = layers.GlobalMaxPool2D()(x)
    x = layers.Dense(128, activation = "relu")(x)

    outputs = layers.Dense(1, activation = "sigmoid")(x)

    model = keras.Model(inputs, outputs)

    return model


input_shape = (256, 256, 3)
model = get_functional_model(input_shape)

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 conv2d_8 (Conv2D)           (None, 256, 256, 64)      1792      
                                                                 
 conv2d_9 (Conv2D)           (None, 256, 256, 64)      36928     
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 128, 128, 64)      0         
 g2D)                                                            
                                                                 
 batch_normalization_2 (Bat  (None, 128, 128, 64)      256       
 chNormalization)                                                
                                                                 
 dropout_2 (Dropout)         (None, 128, 128, 64)      0     

## Model subclassing


In [ ]:
class SimpleCNN(keras.Model):
    def __init__(self):
        super(SimpleCNN, self).__init__()

        self.conv_block1 = keras.Sequential(
            [
                layers.Conv2D(64, 3, strides = 1, activation = 'relu', padding ='same'),
                layers.Conv2D(64, 3, strides = 1, activation = 'relu', padding ='same'),
                layers.MaxPool2D(),
                layers.BatchNormalization(),
                layers.Dropout(0.5),
            ], name = 'conv_block_1'
        )

        self.conv_block2 = keras.Sequential(
            [
                layers.Conv2D(128, 3, strides = 1, activation = 'relu', padding ='same'),
                layers.Conv2D(128, 3, strides = 1, activation = 'relu', padding ='same'),
                layers.MaxPool2D(),
                layers.BatchNormalization(),
                layers.Dropout(0.3),
            ], name = 'conv_block_2'
        )

        self.classfier = keras.Sequential(
            [
                layers.GlobalMaxPool2D(),
                layers.Dense(128, activation = "relu"),
                layers.Dense(1, activation = "sigmoid"),
            ],  name = 'classfier'
        )

    def call(self, input_tensor, training = False):
        x = self.conv_block1(input_tensor)
        x = self.conv_block2(x)
        x = self.classfier(x)

        return x





In [ ]:
input_shape = (None, 256, 256, 3)
model = SimpleCNN()
model.build(input_shape)

model.summary()

Model: "simple_cnn_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv_block_1 (Sequential)   (None, 128, 128, 64)      38976     
                                                                 
 conv_block_2 (Sequential)   (None, 64, 64, 128)       221952    
                                                                 
 classfier (Sequential)      (None, 1)                 16641     
                                                                 
Total params: 277569 (1.06 MB)
Trainable params: 277185 (1.06 MB)
Non-trainable params: 384 (1.50 KB)
_________________________________________________________________


## Model Compiling

In [ ]:
model.compile(
    optimizer = 'adam',
    loss = 'binary_crossentropy',
    metrics = 'accuracy'

)